## BMI

### Model -

### Input - Images

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.image import resize
from sklearn.metrics import r2_score

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
df = pd.read_pickle("/content/drive/MyDrive/myprmlproj/custom_dataset.pkl")

In [11]:
df = df.head(2000)
df.drop(['id','sex','image_side_data'],axis = 1,inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bmi               2000 non-null   float64
 1   image_front_data  2000 non-null   object 
dtypes: float64(1), object(1)
memory usage: 31.4+ KB


In [12]:
df.head()

,bmi,image_front_data
0,24.66,"[185, 184, 183, 182, 181, 181, 181, 183, 183, ..."
1,33.23,"[181, 183, 189, 191, 195, 200, 205, 209, 205, ..."
2,34.21,"[165, 166, 169, 169, 169, 168, 166, 164, 164, ..."
3,28.48,"[160, 154, 154, 155, 151, 149, 150, 153, 156, ..."
4,34.95,"[149, 150, 151, 151, 151, 150, 150, 150, 149, ..."


In [13]:
# Convert the list to a 2D matrix of shape (512, 512)
df['image_front_data'] = df['image_front_data'].apply(lambda x: np.array(x).reshape(512, 512))

In [14]:
# Extract inputs and target
X_front = np.stack(df['image_front_data'].values)  # Convert to NumPy array
y = df['bmi'].values  # Target variable

In [15]:
X_front.shape

(2000, 512, 512)

In [16]:
# Resize images to 128x128
X_front_resized = X_front[..., np.newaxis] / 255.0  # Add channel and normalize
X_front_resized = np.array([resize(img, (128, 128)).numpy() for img in X_front_resized])


In [17]:
X_front_resized.shape

(2000, 128, 128, 1)

In [18]:
# Train-test split
X_front_train, X_front_test, y_train, y_test = train_test_split(
    X_front_resized, y, test_size=0.2, random_state=42
)

In [19]:
# Input shape for resized grayscale images
input_shape = (128, 128, 1)

In [20]:
# Front Image Model
front_input = Input(shape=input_shape, name="front_input")
x = Conv2D(16, (3, 3), activation="relu")(front_input)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation="relu")(x)
output = Dense(1, activation="linear")(x)

In [21]:
# Model definition
model = Model(inputs=front_input, outputs=output)

In [22]:
# Compile the model
model.compile(optimizer="adam", loss="mse", metrics=["mae"])


In [23]:
# Print model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ front_input (InputLayer)             │ (None, 128, 128, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 126, 126, 16)        │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 63504)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       4,064,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,064,545 (15.51 MB)

 Trainable params: 4,064,545 (15.51 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# Train the model
history = model.fit(
    X_front_train, y_train,
    validation_split=0.2,
    epochs=10,  # Reduced epochs
    batch_size=2  # Smaller batch size
)


Epoch 1/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 83.9743 - mae: 7.1291 - val_loss: 37.1008 - val_mae: 4.9257
Epoch 2/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 33.2990 - mae: 4.5158 - val_loss: 34.4425 - val_mae: 4.6268
Epoch 3/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 30.2146 - mae: 4.3527 - val_loss: 34.1570 - val_mae: 4.5597
Epoch 4/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 25.3467 - mae: 3.9337 - val_loss: 35.0345 - val_mae: 4.5786
Epoch 5/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 21.9041 - mae: 3.5643 - val_loss: 35.9787 - val_mae: 4.5422
Epoch 6/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 17.7050 - mae: 3.2437 - val_loss: 37.5386 - val_mae: 4.6375
Epoch 7/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 12.4192 - mae: 2.7479 - val_loss: 40.2119 - val_mae: 4.7687
Epoch 8/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 9.8654 - mae: 2.4572 - val_loss: 40.5191 - val_mae: 4.8446
Epoch 9/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 3

In [25]:
# Evaluate the model
loss, mae = model.evaluate(X_front_test, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 38.1257 - mae: 4.8652
Test Loss: 39.403526306152344, Test MAE: 4.913640022277832


In [26]:
# Predictions on test set
y_pred = model.predict(X_front_test)

# Calculate R² score
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

# Calculate Accuracy within a margin (e.g., within 5 BMI units)
margin = 5
accuracy = np.mean(np.abs(y_test - y_pred.flatten()) <= margin)
print(f"Accuracy within {margin} BMI units: {accuracy * 100:.2f}%")

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
R² Score: -0.24735681359663486
Accuracy within 5 BMI units: 60.25%
